In [47]:
import vk                                                                       
import time                                                                     
import pickle                                                                   
import numpy as np 
import collections
import itertools
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
from collections import defaultdict                                             
                                                                                
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)

In [48]:
all_members = {}
all_members.update(load_obj('./Anya_persons'))
print(len(all_members))
all_members.update(load_obj('./Dasha_persons'))
print(len(all_members))
all_members.update(load_obj('./Maks_persons'))

15209
31037


In [49]:
len(all_members)

46169

In [50]:
users_with_no_inf = []
groups_l = set()
for pers in all_members:
    try:
        groups_l.update(all_members[pers]['groups']) 
    except Exception: users_with_no_inf.append(pers)
groups_l = list(groups_l)

In [51]:
groups = {idx:group for idx, group in enumerate(groups_l)}
reverse_groups = {group:idx for idx, group in enumerate(groups_l)}

In [52]:
print(max(groups.keys()))
print(max(reverse_groups.keys()))

1141946
189802470


In [53]:
for pers in users_with_no_inf:
    del all_members[pers]

In [54]:
users_l = list(all_members.keys())
users = {idx:user for idx, user in enumerate(users_l)}
reverse_users = {user:idx for idx, user in enumerate(users_l)}

In [55]:
data = []
row = []
col =[]
for idx, (user, val) in enumerate(all_members.items()):
    for group in val['groups'][0:int(0.33*len(val['groups']))]:
        row.append(idx)
        data.append(3)
        col.append(reverse_groups[group])
    for group in val['groups'][int(0.33*len(val['groups'])):int(0.66*len(val['groups']))]:
        row.append(idx)
        data.append(2)
        col.append(reverse_groups[group])
    for group in val['groups'][int(0.66*len(val['groups'])):]:
        row.append(idx)
        data.append(1)
        col.append(reverse_groups[group])

In [56]:
csr = csr_matrix((np.array(data), (np.array(row),np.array(col))))

In [57]:
csr.shape

(46169, 1141947)

### Пересчет metrics

In [58]:
from scipy import spatial
from lightfm import LightFM
from sklearn.metrics import pairwise_distances
from scipy.sparse import coo_matrix, csr_matrix, load_npz
from sklearn.metrics.pairwise import cosine_similarity
model = LightFM(loss='warp', no_components=100, learning_rate=0.03, learning_schedule="adadelta")

In [59]:
model.fit(coo_matrix(csr), epochs=5, num_threads=40, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [60]:
user_feature_bias, user_feature_embeddings = model.get_user_representations()
item_feature_bias, item_feature_embeddings = model.get_item_representations()

In [61]:
n_ages=[]
for i in all_members:
    try: n_ages.append(all_members[i]['age'])
    except Exception: continue

In [62]:
mean_age=np.array(n_ages).mean()

In [63]:
sexs=[]
universitys=[]
facultys=[]
ages=[]
citys=[]
for i in all_members:
    try: sexs.append(all_members[i]['sex'])
    except Exception:sexs.append(0)
    try: universitys.append(all_members[i]['university'])
    except Exception:universitys.append(0)
    try: facultys.append(all_members[i]['faculty'])
    except Exception:facultys.append(0)
    try: ages.append(all_members[i]['age'])
    except Exception: ages.append(mean_age)
    try: citys.append(all_members[i]['city'])
    except Exception:citys.append(0)

In [64]:
reverse_citys_d = {city:idx for idx, city in enumerate(set(citys))}
reverse_citys=[reverse_citys_d[citys[i]] for i in range(len(citys))]

In [65]:
reverse_facultys_d = {faculty:idx for idx, faculty in enumerate(set(facultys))}
reverse_facultys=[reverse_facultys_d[facultys[i]] for i in range(len(facultys))]

In [66]:
reverse_universitys_d = {university:idx for idx, university in enumerate(set(universitys))}
reverse_universitys=[reverse_universitys_d[universitys[i]] for i in range(len(universitys))]

In [67]:
import keras
w1=user_feature_embeddings
for i in [sexs,keras.utils.to_categorical(reverse_universitys),keras.utils.to_categorical(reverse_facultys),ages,keras.utils.to_categorical(reverse_citys)]:
    w1=np.hstack((w1,np.vstack(i)))

In [68]:
w1.shape

(46169, 4439)

In [69]:
def recs(idx,csr_a, w):
    # считаем косинус между всеми пользователями
    metrics = cosine_similarity(w[idx].reshape(1, -1), csr_matrix(w)).reshape(-1, 1)
    # домножаем оценки пользовтеля на коэффициент похожести
    #print('metrics', metrics)
    #csr_a *= metrics
    csr_a=csr_a.multiply(metrics)
    total_rate = np.sum(csr_a, axis=0)
    #total_rate=metrics.transpose()*csr_a
    return total_rate

### Рекомендации

In [50]:
reverse_users[9183085]

2794

In [72]:
reverse_users[30811126]

3220

In [119]:
user=3220
t=recs(user, csr, w1)
a=t.tolist()
a = [(j,i) for i , j in enumerate(a[0]) ]
a=sorted(a, key=lambda i: i[0], reverse=True)
k=0
for score, idx in a:
    if k<6:
        if groups[idx] not in all_members[users[user]]['groups']:
            print (groups[idx])
            k=k+1
    else:
        break

0 28986.683516381458 29534144
1 28775.60471950622 65652356
2 24237.459059911645 37119411
3 22440.62622488729 29559271
4 22386.726690660947 30022666
5 21162.749300422096 49866476


In [120]:
def precision_for_one(csr, user_l,w):
    user = reverse_users[user_l]
    t = recs(user, csr, w)
    a = t.tolist()
    a = a[0]
    a = [(j,i) for i , j in enumerate(a)]
    a.sort(key=lambda i: i[0], reverse=True)
    i = a[:6]
    recomend = set([groups[idx] for score, idx in i])
    groups_of_user = set(all_members[user_l]['groups'])
    TP = len(recomend&groups_of_user)
    return TP/6.

In [124]:
import random
users_for_test = random.sample(users_l, 100)

In [125]:
list_of_prec1 = []
k=0
for i in users_for_test:
    print (k, end=' ')
    list_of_prec1.append(precision_for_one(csr, i,w1))
    k=k+1

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [126]:
print(sum(list_of_prec1)/len(list_of_prec1))

0.40333333333333327


In [70]:
with open('Dasha_friends.txt', 'r') as f:
    nums = f.read().splitlines()
print(nums)
with open('Anya_friends.txt', 'r') as f:
    nums1 = f.read().splitlines()
print(nums1)
with open('Maks_friends.txt', 'r') as f:
    nums2 = f.read().splitlines()
print(nums2)

['5179470', '9558686', '17260358', '32670543', '89754899', '484844105', '187961856', '139106107', '182453074', '11311268', '19370392', '67495362']
['59401294', '33436943', '12653364', '300102978', '284227818', '120889121', '310242262', '120942983', '243561432', '267294790']
['995864', '6173127', '6811139', '8431101', '9420850', '10804323', '11030995', '14245770', '17468315', '22840792', '25567189', '25883865', '29145732', '49363671', '62369794', '66382695', '78330794', '171038940', '178159310', '182861903', '292309514', '38628143']


In [71]:
nums3=[30811126, 9183085]

In [72]:
fr=[int(item) for item in nums1+nums2+nums+nums3]

In [73]:
rev_fr=[reverse_users[item] for item in list(set(fr)&set(all_members.keys()))]

In [74]:
all_recs=dict()
i=0
for user in rev_fr:
    print(i, end=' ')
    i=i+1
    dic=dict()
    t=recs(user, csr, w1)
    a=t.tolist()
    a = [(j,i) for i , j in enumerate(a[0]) ]
    a=sorted(a, key=lambda i: i[0], reverse=True)
    k=0
    rec_gr=[]
    for score, idx in a:
        if k<6:
            if groups[idx] not in all_members[users[user]]['groups']:
                #print (k, score, groups[idx])
                rec_gr.append(groups[idx])
                k=k+1
        else:
            break
        dic['groups']=rec_gr
        dic['method']=2
    all_recs[users[user]]=dic
    

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 

In [75]:
all_recs

{187961856: {'groups': [29534144,
   65652356,
   37119411,
   29559271,
   24098496,
   38664418],
  'method': 2},
 120942983: {'groups': [29534144,
   65652356,
   37119411,
   59592700,
   29559271,
   30022666],
  'method': 2},
 14245770: {'groups': [29534144,
   59592700,
   30022666,
   38664418,
   78019879,
   34215577],
  'method': 2},
 292309514: {'groups': [29534144,
   37119411,
   29559271,
   30022666,
   49866476,
   24098496],
  'method': 2},
 33436943: {'groups': [37119411,
   49866476,
   34215577,
   38683579,
   31976785,
   57846937],
  'method': 2},
 19370392: {'groups': [29534144,
   37119411,
   59592700,
   29559271,
   30022666,
   49866476],
  'method': 2},
 995864: {'groups': [29534144,
   65652356,
   37119411,
   59592700,
   29559271,
   30022666],
  'method': 2},
 9558686: {'groups': [29534144,
   65652356,
   37119411,
   29559271,
   30022666,
   49866476],
  'method': 2},
 120889121: {'groups': [37119411,
   59592700,
   29559271,
   30022666,
   2409

In [76]:
with open('Anya_recs.pkl', 'wb') as f:
    pickle.dump(all_recs, f)

In [77]:
recs=load_obj('./Anya_recs')

In [78]:
recs

{187961856: {'groups': [29534144,
   65652356,
   37119411,
   29559271,
   24098496,
   38664418],
  'method': 2},
 120942983: {'groups': [29534144,
   65652356,
   37119411,
   59592700,
   29559271,
   30022666],
  'method': 2},
 14245770: {'groups': [29534144,
   59592700,
   30022666,
   38664418,
   78019879,
   34215577],
  'method': 2},
 292309514: {'groups': [29534144,
   37119411,
   29559271,
   30022666,
   49866476,
   24098496],
  'method': 2},
 33436943: {'groups': [37119411,
   49866476,
   34215577,
   38683579,
   31976785,
   57846937],
  'method': 2},
 19370392: {'groups': [29534144,
   37119411,
   59592700,
   29559271,
   30022666,
   49866476],
  'method': 2},
 995864: {'groups': [29534144,
   65652356,
   37119411,
   59592700,
   29559271,
   30022666],
  'method': 2},
 9558686: {'groups': [29534144,
   65652356,
   37119411,
   29559271,
   30022666,
   49866476],
  'method': 2},
 120889121: {'groups': [37119411,
   59592700,
   29559271,
   30022666,
   2409